# Focault test data reduction

This document will describe step by step the process of extracting the surface profile of a telescope mirror from Foucault or Wire test data.

## Modeling the data

Before jumping into the simulation, we need to find the model that will obtain the mirror surface profile from test data.
After that, we will perform a simulation with real world data.

### Derivation of the equations

First we need to derive the set of equations that will alow to obtaing the mirror surface profile from test data.

For a generic concave mirror surface, the Foucault or Wire tests will show for each mirror radius the equivalent radius of curvature it has. This can be interpreted as finding a succession of spherical surfaces that approximate the existing mirror surface (dashed curve in figure). Obviously that for a completely spherical mirror surface, there is exactly one sphere that matches the whole surface.

<img src="figures/diagram_1.svg"/>

The result of these tests is a set of data pairs consisting of mirror diameter $x$ (independent variable) and its respective radius of curvature $f(x)$ (dependent variable).

To help formulate the equations that model these tests, we can instead think of vectors $G(x)$ and $F(x)$ representing surface height respective to its center and radius of curvature respectively:

$G(x) = \langle x, g(x) \rangle$

$F(x) = \langle 0, f(x) \rangle$

Subtracting the two vectors gives:

$F(x) - G(x) = \langle 0 - x, f(x)-g(x) \rangle = \langle -x, f(x) - g(x) \rangle$

If we then represent the tangent vector to mirror surface at $G(x)$:

$T(x) = \frac{G'(x)}{|G'(x)|} = \langle\frac{1}{|G'(x)|}, \frac{z'(x)}{|G'(x)|}\rangle$

Then it follows that:

$T(x) \cdot (F(x) - G(x)) = 0$

$\left( - \frac{x}{|G'(x)|} + (f(x) - g(x))\frac{g'(x)}{|G'(x)|} \right) = 0$

$(f(x) - g(x))g'(x) = x$

This results in the following differential equation:

$g'(x) = \frac{x}{f(x) - g(x)}$

Knowing the data pairs $x$ and $f(x)$, we can integrate the equation above and get the mirror surface profile.

### Finding a solution

Now in order to ... tbd Runge Kutta

## Real world example

The real world collected data that will be used in this exercise is stored in a JSON file.

First we need to read the test data from the JSON file.

In [1]:
import json

with open('mirror_measurements_database.json', "r") as data_base_file:
    data_base = json.load(data_base_file)
    data_base_file.close()


measurements = sorted(data_base['measurements'], key=lambda m: m['optical_axis_offset'])

For convenience and to allow later use of the SciPy mathematical functions, we will first transform the data into Numpy arrays.

In [2]:
import pandas

measurements_data_frame = pandas.DataFrame(measurements)

display(measurements_data_frame) # Module display is part of library IPython

measurements_data_frame = measurements_data_frame.iloc[:,:2] # Remove the column containing the Foucault image captures

measurements_matrix = measurements_data_frame.to_numpy()
x = measurements_matrix[:, 1]
f = measurements_matrix[:, 0]

,optical_axis_offset,mirror_radius,foucaultgram
0,1.0,5.4,img_1.jpg
1,1.1,10.1,img_3.jpg
2,1.2,20.4,img_3.jpg
3,1.3,30.4,img_3.jpg
4,1.4,40.4,img_3.jpg
5,1.5,50.4,img_3.jpg
6,1.6,60.4,img_3.jpg
7,1.7,70.4,img_3.jpg


kjhkjhkh

In [3]:
from scipy import interpolate
import numpy

interpolation_function = interpolate.interp1d(x, f, kind="cubic")

num_samples = int((x[-1] - x[0])*2)

x_interpolated = numpy.linspace(x[0], x[-1], num=num_samples)
f_interpolated = interpolation_function(x_interpolated)

kjhkhkjh

In [4]:
def aspheric_surface_ode(r, z, f):
    dz = np.divide(r, np.subtract(f, z))
    return dz

In [5]:
from scipy.integrate import solve_ivp


ode_solution = solve_ivp(fun=lambda r, z: aspheric_surface_ode(r, z, test_data_interpolation_function(r)),
                         t_span=[r_sample_points[0], r_sample_points[-1]],
                         y0=[0.0],
                         method="RK45",
                         t_eval=r_sample_points,
                         dense_output=True)

solve_bvp()

NameError: name 'r_sample_points' is not defined

In [ ]:
results = (r_sample_points,
           f_test_data_interpolated_points - self.mirror_details['expected_radius_of_curvature'],
           np.reshape(ode_solution.y, (-1)))